In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("MovieRatingsAnalysis").getOrCreate()

In [ ]:
movies_df = spark.read.csv('' , inferSchema= True , header = True)
ratings_df = spark.read.csv('' , inferSchema = True , header = True)

In [ ]:
movies_rdd = movies_df.rdd
ratings_rdd = ratings_df.rdd

<h1>A</h1>

In [3]:
avg_ratings_rdd = ratings_rdd.map(lambda x: (x['movieId'], (x['rating'], 1)))\
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
.mapValues(lambda x: x[0] / x[1])

NameError: name 'ratings_rdd' is not defined

In [ ]:
lowest_avg_rating = avg_ratings_rdd.sortBy(lambda x: x[1]).first()
print(f"Movie with the lowest average rating: {lowest_avg_rating}"

<h1>B</h1>

In [ ]:
user_ratings_count = ratings_rdd.map(lambda x: (x['userId'], 1))\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda x: x[1], ascending=False)
# Get top users
top_users = user_ratings_count.take(10)
print(f"Top users by number of ratings: {top_users}")

<h1>C</h1>

In [ ]:
from pyspark.sql.functions import from_unixtime, year, month
# Convert timestamp to date and extract year and month
ratings_df = ratings_df.withColumn("year", year(from_unixtime(ratings_df['timestamp'])))\
.withColumn("month", month(from_unixtime(ratings_df['timestamp'])))
# Group by year and month to get rating counts
ratings_over_time = ratings_df.groupBy("year", "month").count().orderBy("year", "month")
ratings_over_time.show()

<h1>D</h1>

In [ ]:
movie_ratings_stats = ratings_rdd.map(lambda x: (x['movieId'], (x['rating'], 1)))\
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
.mapValues(lambda x: (x[0] / x[1], x[1])) # (avg_rating, count)
# Filter movies with a minimum number of ratings
min_ratings = 100
qualified_movies = movie_ratings_stats.filter(lambda x: x[1][1] >= min_ratings)
# Find the highest-rated movies
highest_rated_movies = qualified_movies.sortBy(lambda x: x[1][0], ascending=False).take(10)
print(f"Highest-rated movies with at least {min_ratings} ratings: {highest_rated_movies}")